<a href="https://colab.research.google.com/github/DartagnanFreire/Campeonato-Brasileiro-Previsao-v1.0/blob/previsao.v3.0/Brasileirao_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
#importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from numpy import random

In [62]:
# Leitura da tabela e criação das colunas das rodadas
df = pd.read_excel('/content/brasileirodefinitivo.xlsx')

df.head(10)

,time_mandante,time_visitante,jogos,gols_mandante,gols_visitante
0,América Mineiro,Athletico Paranaense,6,11,4
1,América Mineiro,Atlético Mineiro,5,2,6
2,América Mineiro,Bahia,3,3,1
3,América Mineiro,Botafogo,5,8,2
4,América Mineiro,Corinthians,7,6,5
5,América Mineiro,Coritiba,3,5,4
6,América Mineiro,Cruzeiro,4,2,9
7,América Mineiro,Cuiabá,3,3,3
8,América Mineiro,Flamengo,5,6,9
9,América Mineiro,Fluminense,6,4,4


In [63]:
times = ['Palmeiras', 'Cuiabá', 'América Mineiro', 'Fluminense', 'Botafogo', 'São Paulo', 'Bragantino', 'Bahia', 'Athletico Paranaense', 'Goias',
             'Fortaleza', 'Internacional', 'Atlético Mineiro', 'Vasco da Gama', 'Corinthians', 'Cruzeiro', 'Flamengo','Coritiba', 'Gremio', 'Santos']
tabela1 = pd.DataFrame(times, columns=['Time'])
tabela2 = pd.DataFrame(times, columns=['Time'])
tabela3 = pd.DataFrame(times, columns=['Time'])
tabela4 = pd.DataFrame(times, columns=['Time'])
tabela5 = pd.DataFrame(times, columns=['Time'])
tabela6 = pd.DataFrame(times, columns=['Time'])
tabela7 = pd.DataFrame(times, columns=['Time'])
tabela8 = pd.DataFrame(times, columns=['Time'])
tabela9 = pd.DataFrame(times, columns=['Time'])
tabela10 = pd.DataFrame(times, columns=['Time'])

In [64]:
# Criação da função. Parametros: tabela dos dados, time mandante, time visitante, rodada
def times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, home_team, away_team, rodada):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import poisson
    from numpy import random

    # Função de simulação do mandante
    def mandante_prob(n):
        gols = 0
        prob = 0
        for i in range(0, 1000000):
            if mandante_poisson[i] == n:
                gols += 1
                prob = gols / 1000000
        return prob, gols


    # Função de simulação do visitante
    def visitante_prob(n):
        gols = 0
        prob = 0
        for i in range(0,1000000):
            if visitante_poisson[i] == n:
                gols += 1
                prob = gols / 1000000
        return prob, gols


    # Função de soma dos resultados
    def somas(mat, n):
        principal = 0
        secundario = 0
        for i in range(0, n):
            for j in range(0, n):
                if (i==j):
                    principal += mat[i][j]
                if ((i+j) == (n-1)):
                    secundario += mat[i][j]
        return principal

    # Linha do jogo em questão
    linha = df[(df['time_mandante'] == home_team) & (df['time_visitante'] == away_team)]

    # Média de gols do time Mandante todos os jogos
    mandante = df[(df['time_mandante'] == home_team)]
    mm_gols = mandante['gols_mandante'].sum() / mandante['jogos'].sum()

    # Média de gols do time visitante todos os jogos
    visitante = df[(df['time_mandante'] == away_team)]
    mv_gols = visitante['gols_visitante'].sum() / visitante['jogos'].sum()

    # Média de gols do time home_team
    home_feitos = linha['gols_mandante'].sum() / linha['jogos'].sum()

    # Média de gols cedidos home_team
    home_sofrido = linha['gols_visitante'].sum() / linha['jogos'].sum()

    # Força de Ataque Mandante
    atk_mandante = home_feitos / mm_gols

    # Força de Defesa Mandante
    def_mandante = home_sofrido / mv_gols

    # Força de Ataque Visitante
    atk_visitante = home_sofrido / mv_gols

    # Força de Defesa Visitante
    def_visitante = home_feitos / mm_gols

    # xG Mandante
    xG_mandante = atk_mandante * def_visitante * mm_gols
    xG_visitante = atk_visitante * def_mandante * mv_gols

    # Realizando Poisson para mandante e visitante
    mandante_poisson = random.poisson(lam=xG_mandante, size=1000000)
    visitante_poisson = random.poisson(lam=xG_visitante, size=1000000)

    # Simulando probabilidade de gols mandante
    home_0, g = mandante_prob(0)
    home_1, g = mandante_prob(1)
    home_2, g = mandante_prob(2)
    home_3, g = mandante_prob(3)
    home_4, g = mandante_prob(4)
    home_5, g = mandante_prob(5)

    # Simulando probabilidade de gols visitante
    away_0, g = visitante_prob(0)
    away_1, g = visitante_prob(1)
    away_2, g = visitante_prob(2)
    away_3, g = visitante_prob(3)
    away_4, g = visitante_prob(4)
    away_5, g = visitante_prob(5)

    # Gravando em uma tabela mandante
    mandante_chance = [home_0, home_1, home_2, home_3, home_4, home_5]
    home_chance_tab = pd.DataFrame(mandante_chance, columns=['Probs'])

    # Gravando em uma tabela visitante
    visitante_chance = [away_0, away_1, away_2, away_3, away_4, away_5]
    visitante_chance_tab = pd.DataFrame(visitante_chance, columns=['Probs'])

    # Montando uma tabela com os dados cruzados, e arrumando a porcentagem
    df_cross = home_chance_tab.dot(visitante_chance_tab.T)
    df_cross = df_cross.round(3)
    df_cross[0] = df_cross[0]*100
    df_cross[1] = df_cross[1]*100
    df_cross[2] = df_cross[2]*100
    df_cross[3] = df_cross[3]*100
    df_cross[4] = df_cross[4]*100
    df_cross[5] = df_cross[5]*100

   # Pegando os 5 maiores resultados, em ordem
    lista_valores = []
    for col_index, col_values in df_cross.items():
        for i, valor in enumerate(col_values):
            lista_valores.append((col_index, i, valor))

    # Ordena a lista do maior para o menor
    lista_valores.sort(key=lambda x: x[2], reverse=True)

    # Para obter os 10 maiores valores, com índice e coluna

    cinco = lista_valores[:10]

    # Mostrando os resultados
    #for col_index, row_index, valor in cinco:
        #print(f"{home_team} {row_index}x{col_index} {away_team}: {valor:.2f}%")

    # Somando todos os empates
    principal = somas(df_cross, 5)
    df_cross_up= df_cross.where(np.triu(np.ones(df_cross.shape)).astype(np.bool_))

    # Somando todas vitórias mandante e todas vitórias visitante
    vitoria_mandante = df_cross.sum().sum() - df_cross_up.sum().sum()
    vitoria_visitante = df_cross_up.sum().sum() - principal

    # Mostrando os resultados
    #print(f'Vitoria {home_team}: {vitoria_mandante:.2f}%')
    #print(f'Empate: {principal:.2f}%')
    #print(f'Vitória {away_team}: {vitoria_visitante:.2f}%')

    # Grava os resultados na tabela 1
    if cinco[0][1] > cinco[0][0]:
      tabela1.loc[tabela1.Time == home_team, rodada] = 3

    if cinco[0][1] == cinco[0][0]:
      tabela1.loc[tabela1.Time == home_team, rodada] = 1
      tabela1.loc[tabela1.Time == away_team, rodada] = 1

    if cinco[0][1] < cinco[0][0]:
      tabela1.loc[tabela1.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 2
    if cinco[1][1] > cinco[1][0]:
      tabela2.loc[tabela2.Time == home_team, rodada] = 3

    if cinco[1][1] == cinco[1][0]:
      tabela2.loc[tabela2.Time == home_team, rodada] = 1
      tabela2.loc[tabela2.Time == away_team, rodada] = 1

    if cinco[1][1] < cinco[1][0]:
      tabela2.loc[tabela2.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 3
    if cinco[2][1] > cinco[2][0]:
      tabela3.loc[tabela3.Time == home_team, rodada] = 3

    if cinco[2][1] == cinco[2][0]:
      tabela3.loc[tabela3.Time == home_team, rodada] = 1
      tabela3.loc[tabela3.Time == away_team, rodada] = 1

    if cinco[2][1] < cinco[2][0]:
      tabela3.loc[tabela3.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 4
    if cinco[3][1] > cinco[3][0]:
      tabela4.loc[tabela4.Time == home_team, rodada] = 3

    if cinco[3][1] == cinco[3][0]:
      tabela4.loc[tabela4.Time == home_team, rodada] = 1
      tabela4.loc[tabela4.Time == away_team, rodada] = 1

    if cinco[3][1] < cinco[3][0]:
      tabela4.loc[tabela4.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 5
    if cinco[4][1] > cinco[4][0]:
      tabela5.loc[tabela5.Time == home_team, rodada] = 3

    if cinco[4][1] == cinco[4][0]:
      tabela5.loc[tabela5.Time == home_team, rodada] = 1
      tabela5.loc[tabela5.Time == away_team, rodada] = 1

    if cinco[4][1] < cinco[4][0]:
      tabela5.loc[tabela5.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 6
    if cinco[5][1] > cinco[5][0]:
      tabela6.loc[tabela6.Time == home_team, rodada] = 3

    if cinco[5][1] == cinco[5][0]:
      tabela6.loc[tabela6.Time == home_team, rodada] = 1
      tabela6.loc[tabela6.Time == away_team, rodada] = 1

    if cinco[5][1] < cinco[5][0]:
      tabela6.loc[tabela6.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 7
    if cinco[6][1] > cinco[6][0]:
      tabela7.loc[tabela7.Time == home_team, rodada] = 3

    if cinco[6][1] == cinco[6][0]:
      tabela7.loc[tabela7.Time == home_team, rodada] = 1
      tabela7.loc[tabela7.Time == away_team, rodada] = 1

    if cinco[6][1] < cinco[6][0]:
      tabela7.loc[tabela7.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 8
    if cinco[7][1] > cinco[7][0]:
      tabela8.loc[tabela8.Time == home_team, rodada] = 3

    if cinco[7][1] == cinco[7][0]:
      tabela8.loc[tabela8.Time == home_team, rodada] = 1
      tabela8.loc[tabela8.Time == away_team, rodada] = 1

    if cinco[7][1] < cinco[7][0]:
      tabela8.loc[tabela8.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 9
    if cinco[8][1] > cinco[8][0]:
      tabela9.loc[tabela9.Time == home_team, rodada] = 3

    if cinco[8][1] == cinco[8][0]:
      tabela9.loc[tabela9.Time == home_team, rodada] = 1
      tabela9.loc[tabela9.Time == away_team, rodada] = 1

    if cinco[8][1] < cinco[8][0]:
      tabela9.loc[tabela9.Time == away_team, rodada] = 3

    # Grava os resultados na tabela 10
    if cinco[9][1] > cinco[9][0]:
      tabela10.loc[tabela10.Time == home_team, rodada] = 3

    if cinco[9][1] == cinco[9][0]:
      tabela10.loc[tabela10.Time == home_team, rodada] = 1
      tabela10.loc[tabela10.Time == away_team, rodada] = 1

    if cinco[9][1] < cinco[9][0]:
      tabela10.loc[tabela10.Time == away_team, rodada] = 3


# Rodadas

In [65]:
# Rodada 1

df['Rodada 1'] = pd.Series(dtype='int')
tabela1['Rodada 1'] = pd.Series(dtype='int')
tabela2['Rodada 1'] = pd.Series(dtype='int')
tabela3['Rodada 1'] = pd.Series(dtype='int')
tabela4['Rodada 1'] = pd.Series(dtype='int')
tabela5['Rodada 1'] = pd.Series(dtype='int')
tabela6['Rodada 1'] = pd.Series(dtype='int')
tabela7['Rodada 1'] = pd.Series(dtype='int')
tabela8['Rodada 1'] = pd.Series(dtype='int')
tabela9['Rodada 1'] = pd.Series(dtype='int')
tabela10['Rodada 1'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Cuiabá', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Fluminense', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'São Paulo', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Bahia', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Goias', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Internacional', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Vasco da Gama', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Cruzeiro', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Coritiba', 'Rodada 1')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Santos', 'Rodada 1')

In [66]:
# Rodada 2

tabela1['Rodada 2'] = pd.Series(dtype='int')
tabela2['Rodada 2'] = pd.Series(dtype='int')
tabela3['Rodada 2'] = pd.Series(dtype='int')
tabela4['Rodada 2'] = pd.Series(dtype='int')
tabela5['Rodada 2'] = pd.Series(dtype='int')
tabela6['Rodada 2'] = pd.Series(dtype='int')
tabela7['Rodada 2'] = pd.Series(dtype='int')
tabela8['Rodada 2'] = pd.Series(dtype='int')
tabela9['Rodada 2'] = pd.Series(dtype='int')
tabela10['Rodada 2'] = pd.Series(dtype='int')
df['Rodada 2'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Athletico Paranaense', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'América Mineiro', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Bragantino', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Gremio', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Flamengo', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Atlético Mineiro', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Palmeiras', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Fortaleza', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Corinthians', 'Rodada 2')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Botafogo', 'Rodada 2')


In [67]:
# Rodada 3

tabela1['Rodada 3'] = pd.Series(dtype='int')
tabela2['Rodada 3'] = pd.Series(dtype='int')
tabela3['Rodada 3'] = pd.Series(dtype='int')
tabela4['Rodada 3'] = pd.Series(dtype='int')
tabela5['Rodada 3'] = pd.Series(dtype='int')
tabela6['Rodada 3'] = pd.Series(dtype='int')
tabela7['Rodada 3'] = pd.Series(dtype='int')
tabela8['Rodada 3'] = pd.Series(dtype='int')
tabela9['Rodada 3'] = pd.Series(dtype='int')
tabela10['Rodada 3'] = pd.Series(dtype='int')
df['Rodada 3'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Fluminense', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'São Paulo', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Goias', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Corinthians', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'América Mineiro', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Cruzeiro', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Athletico Paranaense', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Botafogo', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Bahia', 'Rodada 3')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Gremio', 'Rodada 3')

In [68]:
# Rodada 4

tabela1 ['Rodada 4'] = pd.Series(dtype='int')
tabela2 ['Rodada 4'] = pd.Series(dtype='int')
tabela3 ['Rodada 4'] = pd.Series(dtype='int')
tabela4 ['Rodada 4'] = pd.Series(dtype='int')
tabela5 ['Rodada 4'] = pd.Series(dtype='int')
tabela6 ['Rodada 4'] = pd.Series(dtype='int')
tabela7 ['Rodada 4'] = pd.Series(dtype='int')
tabela8 ['Rodada 4'] = pd.Series(dtype='int')
tabela9 ['Rodada 4'] = pd.Series(dtype='int')
tabela10['Rodada 4'] = pd.Series(dtype='int')
df['Rodada 4'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Santos', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Vasco da Gama', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Cuiabá', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Coritiba', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Internacional', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Flamengo', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Bragantino', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Atlético Mineiro', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Palmeiras', 'Rodada 4')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Fortaleza', 'Rodada 4')

In [69]:
# Rodada 5

tabela1 ['Rodada 5'] = pd.Series(dtype='int')
tabela2 ['Rodada 5'] = pd.Series(dtype='int')
tabela3 ['Rodada 5'] = pd.Series(dtype='int')
tabela4 ['Rodada 5'] = pd.Series(dtype='int')
tabela5 ['Rodada 5'] = pd.Series(dtype='int')
tabela6 ['Rodada 5'] = pd.Series(dtype='int')
tabela7 ['Rodada 5'] = pd.Series(dtype='int')
tabela8 ['Rodada 5'] = pd.Series(dtype='int')
tabela9 ['Rodada 5'] = pd.Series(dtype='int')
tabela10['Rodada 5'] = pd.Series(dtype='int')
df['Rodada 5'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Athletico Paranaense', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Bahia', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'América Mineiro', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Atlético Mineiro', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Goias', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Fluminense', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Gremio', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Vasco da Gama', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Corinthians', 'Rodada 5')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'São Paulo', 'Rodada 5')

In [70]:
# Rodada 6

tabela1 ['Rodada 6'] = pd.Series(dtype='int')
tabela2 ['Rodada 6'] = pd.Series(dtype='int')
tabela3 ['Rodada 6'] = pd.Series(dtype='int')
tabela4 ['Rodada 6'] = pd.Series(dtype='int')
tabela5 ['Rodada 6'] = pd.Series(dtype='int')
tabela6 ['Rodada 6'] = pd.Series(dtype='int')
tabela7 ['Rodada 6'] = pd.Series(dtype='int')
tabela8 ['Rodada 6'] = pd.Series(dtype='int')
tabela9 ['Rodada 6'] = pd.Series(dtype='int')
tabela10['Rodada 6'] = pd.Series(dtype='int')
df['Rodada 6'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Flamengo', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Cuiabá', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Bragantino', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Internacional', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Fortaleza', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Santos', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'São Paulo', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Coritiba', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Botafogo', 'Rodada 6')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Cruzeiro', 'Rodada 6')

In [71]:
# Rodada 7

tabela1 ['Rodada 7'] = pd.Series(dtype='int')
tabela2 ['Rodada 7'] = pd.Series(dtype='int')
tabela3 ['Rodada 7'] = pd.Series(dtype='int')
tabela4 ['Rodada 7'] = pd.Series(dtype='int')
tabela5 ['Rodada 7'] = pd.Series(dtype='int')
tabela6 ['Rodada 7'] = pd.Series(dtype='int')
tabela7 ['Rodada 7'] = pd.Series(dtype='int')
tabela8 ['Rodada 7'] = pd.Series(dtype='int')
tabela9 ['Rodada 7'] = pd.Series(dtype='int')
tabela10['Rodada 7'] = pd.Series(dtype='int')
df['Rodada 7'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Goias', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Athletico Paranaense', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Vasco da Gama', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Atlético Mineiro', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Fortaleza', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Fluminense', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Palmeiras', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Corinthians', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Internacional', 'Rodada 7')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Cuiabá', 'Rodada 7')

In [72]:
# Rodada 8

tabela1 ['Rodada 8'] = pd.Series(dtype='int')
tabela2 ['Rodada 8'] = pd.Series(dtype='int')
tabela3 ['Rodada 8'] = pd.Series(dtype='int')
tabela4 ['Rodada 8'] = pd.Series(dtype='int')
tabela5 ['Rodada 8'] = pd.Series(dtype='int')
tabela6 ['Rodada 8'] = pd.Series(dtype='int')
tabela7 ['Rodada 8'] = pd.Series(dtype='int')
tabela8 ['Rodada 8'] = pd.Series(dtype='int')
tabela9 ['Rodada 8'] = pd.Series(dtype='int')
tabela10['Rodada 8'] = pd.Series(dtype='int')
df['Rodada 8'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Gremio', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Vasco da Gama', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Cruzeiro', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Coritiba', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Goias', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Fluminense', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Bahia', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Palmeiras', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Santos', 'Rodada 8')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'América Mineiro', 'Rodada 8')

In [73]:
# Rodada 9

tabela1['Rodada 9'] = pd.Series(dtype='int')
tabela2['Rodada 9'] = pd.Series(dtype='int')
tabela3['Rodada 9'] = pd.Series(dtype='int')
tabela4['Rodada 9'] = pd.Series(dtype='int')
tabela5['Rodada 9'] = pd.Series(dtype='int')
tabela6['Rodada 9'] = pd.Series(dtype='int')
tabela7['Rodada 9'] = pd.Series(dtype='int')
tabela8['Rodada 9'] = pd.Series(dtype='int')
tabela9['Rodada 9'] = pd.Series(dtype='int')
tabela10['Rodada 9'] = pd.Series(dtype='int')
df['Rodada 9'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Bahia', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Botafogo', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Atlético Mineiro', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Corinthians', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Internacional', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Bragantino', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'São Paulo', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Cuiabá', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Coritiba', 'Rodada 9')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Flamengo', 'Rodada 9')

In [74]:
# Rodada 10

tabela1['Rodada 10'] = pd.Series(dtype='int')
tabela2['Rodada 10'] = pd.Series(dtype='int')
tabela3['Rodada 10'] = pd.Series(dtype='int')
tabela4['Rodada 10'] = pd.Series(dtype='int')
tabela5['Rodada 10'] = pd.Series(dtype='int')
tabela6['Rodada 10'] = pd.Series(dtype='int')
tabela7['Rodada 10'] = pd.Series(dtype='int')
tabela8['Rodada 10'] = pd.Series(dtype='int')
tabela9['Rodada 10'] = pd.Series(dtype='int')
tabela10['Rodada 10'] = pd.Series(dtype='int')
df['Rodada 10'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Santos', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Cruzeiro', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Cuiabá', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Bragantino', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Fortaleza', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Athletico Paranaense', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Palmeiras', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Vasco da Gama', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Gremio', 'Rodada 10')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Fluminense', 'Rodada 10')

In [75]:
# Rodada 11

tabela1['Rodada 11'] = pd.Series(dtype='int')
tabela2['Rodada 11'] = pd.Series(dtype='int')
tabela3['Rodada 11'] = pd.Series(dtype='int')
tabela4['Rodada 11'] = pd.Series(dtype='int')
tabela5['Rodada 11'] = pd.Series(dtype='int')
tabela6['Rodada 11'] = pd.Series(dtype='int')
tabela7['Rodada 11'] = pd.Series(dtype='int')
tabela8['Rodada 11'] = pd.Series(dtype='int')
tabela9['Rodada 11'] = pd.Series(dtype='int')
tabela10['Rodada 11'] = pd.Series(dtype='int')
df['Rodada 11'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Fortaleza', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Athletico Paranaense', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Corinthians', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Atlético Mineiro', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Palmeiras', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'América Mineiro', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Botafogo', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Goias', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Internacional', 'Rodada 11')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Flamengo', 'Rodada 11')

In [76]:
# Rodada 12

tabela1['Rodada 12'] = pd.Series(dtype='int')
tabela2['Rodada 12'] = pd.Series(dtype='int')
tabela3['Rodada 12'] = pd.Series(dtype='int')
tabela4['Rodada 12'] = pd.Series(dtype='int')
tabela5['Rodada 12'] = pd.Series(dtype='int')
tabela6['Rodada 12'] = pd.Series(dtype='int')
tabela7['Rodada 12'] = pd.Series(dtype='int')
tabela8['Rodada 12'] = pd.Series(dtype='int')
tabela9['Rodada 12'] = pd.Series(dtype='int')
tabela10['Rodada 12'] = pd.Series(dtype='int')
df['Rodada 12'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Corinthians', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Bahia', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Atlético Mineiro', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'São Paulo', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Botafogo', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Coritiba', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Internacional', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Flamengo', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Goias', 'Rodada 12')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Cuiabá', 'Rodada 12')

In [77]:
# Rodada 13

tabela1['Rodada 13'] = pd.Series(dtype='int')
tabela2['Rodada 13'] = pd.Series(dtype='int')
tabela3['Rodada 13'] = pd.Series(dtype='int')
tabela4['Rodada 13'] = pd.Series(dtype='int')
tabela5['Rodada 13'] = pd.Series(dtype='int')
tabela6['Rodada 13'] = pd.Series(dtype='int')
tabela7['Rodada 13'] = pd.Series(dtype='int')
tabela8['Rodada 13'] = pd.Series(dtype='int')
tabela9['Rodada 13'] = pd.Series(dtype='int')
tabela10['Rodada 13'] = pd.Series(dtype='int')
df['Rodada 13'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Fluminense', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Fortaleza', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Gremio', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Cruzeiro', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Bragantino', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Vasco da Gama', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'América Mineiro', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Palmeiras', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Santos', 'Rodada 13')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Coritiba', 'Rodada 13')

In [78]:
# Rodada 14

tabela1['Rodada 14'] = pd.Series(dtype='int')
tabela2['Rodada 14'] = pd.Series(dtype='int')
tabela3['Rodada 14'] = pd.Series(dtype='int')
tabela4['Rodada 14'] = pd.Series(dtype='int')
tabela5['Rodada 14'] = pd.Series(dtype='int')
tabela6['Rodada 14'] = pd.Series(dtype='int')
tabela7['Rodada 14'] = pd.Series(dtype='int')
tabela8['Rodada 14'] = pd.Series(dtype='int')
tabela9['Rodada 14'] = pd.Series(dtype='int')
tabela10['Rodada 14'] = pd.Series(dtype='int')
df['Rodada 14'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Bahia', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Cruzeiro', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Corinthians', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'América Mineiro', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Flamengo', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Goias', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Internacional', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'São Paulo', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Athletico Paranaense', 'Rodada 14')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Botafogo', 'Rodada 14')

In [79]:
# Rodada 15

tabela1['Rodada 15'] = pd.Series(dtype='int')
tabela2['Rodada 15'] = pd.Series(dtype='int')
tabela3['Rodada 15'] = pd.Series(dtype='int')
tabela4['Rodada 15'] = pd.Series(dtype='int')
tabela5['Rodada 15'] = pd.Series(dtype='int')
tabela6['Rodada 15'] = pd.Series(dtype='int')
tabela7['Rodada 15'] = pd.Series(dtype='int')
tabela8['Rodada 15'] = pd.Series(dtype='int')
tabela9['Rodada 15'] = pd.Series(dtype='int')
tabela10['Rodada 15'] = pd.Series(dtype='int')
df['Rodada 15'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Vasco da Gama', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Gremio', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Bragantino', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Coritiba', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Cuiabá', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Flamengo', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Santos', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Bahia', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Palmeiras', 'Rodada 15')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Atlético Mineiro', 'Rodada 15')

In [80]:
# Rodada 16

tabela1['Rodada 16'] = pd.Series(dtype='int')
tabela2['Rodada 16'] = pd.Series(dtype='int')
tabela3['Rodada 16'] = pd.Series(dtype='int')
tabela4['Rodada 16'] = pd.Series(dtype='int')
tabela5['Rodada 16'] = pd.Series(dtype='int')
tabela6['Rodada 16'] = pd.Series(dtype='int')
tabela7['Rodada 16'] = pd.Series(dtype='int')
tabela8['Rodada 16'] = pd.Series(dtype='int')
tabela9['Rodada 16'] = pd.Series(dtype='int')
tabela10['Rodada 16'] = pd.Series(dtype='int')
df['Rodada 16'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Fortaleza', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'América Mineiro', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'São Paulo', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Corinthians', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Atlético Mineiro', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Internacional', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Botafogo', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Goias', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Athletico Paranaense', 'Rodada 16')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Fluminense', 'Rodada 16')

In [81]:
# Rodada 17

tabela1['Rodada 17'] = pd.Series(dtype='int')
tabela2['Rodada 17'] = pd.Series(dtype='int')
tabela3['Rodada 17'] = pd.Series(dtype='int')
tabela4['Rodada 17'] = pd.Series(dtype='int')
tabela5['Rodada 17'] = pd.Series(dtype='int')
tabela6['Rodada 17'] = pd.Series(dtype='int')
tabela7['Rodada 17'] = pd.Series(dtype='int')
tabela8['Rodada 17'] = pd.Series(dtype='int')
tabela9['Rodada 17'] = pd.Series(dtype='int')
tabela10['Rodada 17'] = pd.Series(dtype='int')
df['Rodada 17'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Santos', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Cuiabá', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Cruzeiro', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Vasco da Gama', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Bragantino', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Flamengo', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Bahia', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Palmeiras', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Coritiba', 'Rodada 17')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Gremio', 'Rodada 17')

In [82]:
# Rodada 18

tabela1['Rodada 18'] = pd.Series(dtype='int')
tabela2['Rodada 18'] = pd.Series(dtype='int')
tabela3['Rodada 18'] = pd.Series(dtype='int')
tabela4['Rodada 18'] = pd.Series(dtype='int')
tabela5['Rodada 18'] = pd.Series(dtype='int')
tabela6['Rodada 18'] = pd.Series(dtype='int')
tabela7['Rodada 18'] = pd.Series(dtype='int')
tabela8['Rodada 18'] = pd.Series(dtype='int')
tabela9['Rodada 18'] = pd.Series(dtype='int')
tabela10['Rodada 18'] = pd.Series(dtype='int')
df['Rodada 18'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Athletico Paranaense', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'América Mineiro', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Corinthians', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Palmeiras', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Bragantino', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Fortaleza', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Gremio', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Atlético Mineiro', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Botafogo', 'Rodada 18')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Flamengo', 'Rodada 18')

In [83]:
# Rodada 19

tabela1['Rodada 19'] = pd.Series(dtype='int')
tabela2['Rodada 19'] = pd.Series(dtype='int')
tabela3['Rodada 19'] = pd.Series(dtype='int')
tabela4['Rodada 19'] = pd.Series(dtype='int')
tabela5['Rodada 19'] = pd.Series(dtype='int')
tabela6['Rodada 19'] = pd.Series(dtype='int')
tabela7['Rodada 19'] = pd.Series(dtype='int')
tabela8['Rodada 19'] = pd.Series(dtype='int')
tabela9['Rodada 19'] = pd.Series(dtype='int')
tabela10['Rodada 19'] = pd.Series(dtype='int')
df['Rodada 19'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Goias', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Cruzeiro', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Internacional', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Bahia', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Coritiba', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Fluminense', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'São Paulo', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Santos', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Cuiabá', 'Rodada 19')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Vasco da Gama', 'Rodada 19')

In [84]:
# Rodada 20

tabela1['Rodada 20'] = pd.Series(dtype='int')
tabela2['Rodada 20'] = pd.Series(dtype='int')
tabela3['Rodada 20'] = pd.Series(dtype='int')
tabela4['Rodada 20'] = pd.Series(dtype='int')
tabela5['Rodada 20'] = pd.Series(dtype='int')
tabela6['Rodada 20'] = pd.Series(dtype='int')
tabela7['Rodada 20'] = pd.Series(dtype='int')
tabela8['Rodada 20'] = pd.Series(dtype='int')
tabela9['Rodada 20'] = pd.Series(dtype='int')
tabela10['Rodada 20'] = pd.Series(dtype='int')
df['Rodada 20'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Botafogo', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Palmeiras', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'América Mineiro', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Corinthians', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Atlético Mineiro', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Flamengo', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Bragantino', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Gremio', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Athletico Paranaense', 'Rodada 20')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Fortaleza', 'Rodada 20')

In [85]:
# Rodada 21

tabela1['Rodada 21'] = pd.Series(dtype='int')
tabela2['Rodada 21'] = pd.Series(dtype='int')
tabela3['Rodada 21'] = pd.Series(dtype='int')
tabela4['Rodada 21'] = pd.Series(dtype='int')
tabela5['Rodada 21'] = pd.Series(dtype='int')
tabela6['Rodada 21'] = pd.Series(dtype='int')
tabela7['Rodada 21'] = pd.Series(dtype='int')
tabela8['Rodada 21'] = pd.Series(dtype='int')
tabela9['Rodada 21'] = pd.Series(dtype='int')
tabela10['Rodada 21'] = pd.Series(dtype='int')
df['Rodada 21'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Vasco da Gama', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Internacional', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Cuiabá', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Cruzeiro', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Bahia', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Santos', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Coritiba', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Goias', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Fluminense', 'Rodada 21')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'São Paulo', 'Rodada 21')

In [86]:
# Rodada 22

tabela1['Rodada 22'] = pd.Series(dtype='int')
tabela2['Rodada 22'] = pd.Series(dtype='int')
tabela3['Rodada 22'] = pd.Series(dtype='int')
tabela4['Rodada 22'] = pd.Series(dtype='int')
tabela5['Rodada 22'] = pd.Series(dtype='int')
tabela6['Rodada 22'] = pd.Series(dtype='int')
tabela7['Rodada 22'] = pd.Series(dtype='int')
tabela8['Rodada 22'] = pd.Series(dtype='int')
tabela9['Rodada 22'] = pd.Series(dtype='int')
tabela10['Rodada 22'] = pd.Series(dtype='int')
df['Rodada 22'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Flamengo', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Vasco da Gama', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Internacional', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Cuiabá', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Bragantino', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Palmeiras', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Coritiba', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Atlético Mineiro', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Santos', 'Rodada 22')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Fortaleza', 'Rodada 22')

In [87]:
# Rodada 23

tabela1['Rodada 23'] = pd.Series(dtype='int')
tabela2['Rodada 23'] = pd.Series(dtype='int')
tabela3['Rodada 23'] = pd.Series(dtype='int')
tabela4['Rodada 23'] = pd.Series(dtype='int')
tabela5['Rodada 23'] = pd.Series(dtype='int')
tabela6['Rodada 23'] = pd.Series(dtype='int')
tabela7['Rodada 23'] = pd.Series(dtype='int')
tabela8['Rodada 23'] = pd.Series(dtype='int')
tabela9['Rodada 23'] = pd.Series(dtype='int')
tabela10['Rodada 23'] = pd.Series(dtype='int')
df['Rodada 23'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Athletico Paranaense', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Cruzeiro', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'São Paulo', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Fluminense', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Goias', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Gremio', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Corinthians', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'América Mineiro', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Bahia', 'Rodada 23')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Botafogo', 'Rodada 23')

In [88]:
# Rodada 24

tabela1['Rodada 24'] = pd.Series(dtype='int')
tabela2['Rodada 24'] = pd.Series(dtype='int')
tabela3['Rodada 24'] = pd.Series(dtype='int')
tabela4['Rodada 24'] = pd.Series(dtype='int')
tabela5['Rodada 24'] = pd.Series(dtype='int')
tabela6['Rodada 24'] = pd.Series(dtype='int')
tabela7['Rodada 24'] = pd.Series(dtype='int')
tabela8['Rodada 24'] = pd.Series(dtype='int')
tabela9['Rodada 24'] = pd.Series(dtype='int')
tabela10['Rodada 24'] = pd.Series(dtype='int')
df['Rodada 24'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Bragantino', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Internacional', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Santos', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Cruzeiro', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Flamengo', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Fortaleza', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Botafogo', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Cuiabá', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Palmeiras', 'Rodada 24')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Coritiba', 'Rodada 24')

In [89]:
# Rodada 25

tabela1['Rodada 25'] = pd.Series(dtype='int')
tabela2['Rodada 25'] = pd.Series(dtype='int')
tabela3['Rodada 25'] = pd.Series(dtype='int')
tabela4['Rodada 25'] = pd.Series(dtype='int')
tabela5['Rodada 25'] = pd.Series(dtype='int')
tabela6['Rodada 25'] = pd.Series(dtype='int')
tabela7['Rodada 25'] = pd.Series(dtype='int')
tabela8['Rodada 25'] = pd.Series(dtype='int')
tabela9['Rodada 25'] = pd.Series(dtype='int')
tabela10['Rodada 25'] = pd.Series(dtype='int')
df['Rodada 25'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Corinthians', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'América Mineiro', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Fluminense', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Athletico Paranaense', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Vasco da Gama', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Bahia', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Goias', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Palmeiras', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Gremio', 'Rodada 25')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Atlético Mineiro', 'Rodada 25')

In [90]:
# Rodada 26

tabela1['Rodada 26'] = pd.Series(dtype='int')
tabela2['Rodada 26'] = pd.Series(dtype='int')
tabela3['Rodada 26'] = pd.Series(dtype='int')
tabela4['Rodada 26'] = pd.Series(dtype='int')
tabela5['Rodada 26'] = pd.Series(dtype='int')
tabela6['Rodada 26'] = pd.Series(dtype='int')
tabela7['Rodada 26'] = pd.Series(dtype='int')
tabela8['Rodada 26'] = pd.Series(dtype='int')
tabela9['Rodada 26'] = pd.Series(dtype='int')
tabela10['Rodada 26'] = pd.Series(dtype='int')
df['Rodada 26'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Gremio', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'São Paulo', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Bragantino', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Coritiba', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Santos', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Bahia', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Botafogo', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Cruzeiro', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'América Mineiro', 'Rodada 26')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Flamengo', 'Rodada 26')

In [91]:
# Rodada 27

tabela1['Rodada 27'] = pd.Series(dtype='int')
tabela2['Rodada 27'] = pd.Series(dtype='int')
tabela3['Rodada 27'] = pd.Series(dtype='int')
tabela4['Rodada 27'] = pd.Series(dtype='int')
tabela5['Rodada 27'] = pd.Series(dtype='int')
tabela6['Rodada 27'] = pd.Series(dtype='int')
tabela7['Rodada 27'] = pd.Series(dtype='int')
tabela8['Rodada 27'] = pd.Series(dtype='int')
tabela9['Rodada 27'] = pd.Series(dtype='int')
tabela10['Rodada 27'] = pd.Series(dtype='int')
df['Rodada 27'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'São Paulo', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Atlético Mineiro', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Corinthians', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Internacional', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Flamengo', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Fortaleza', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Athletico Paranaense', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Botafogo', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Bragantino', 'Rodada 27')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Cuiabá', 'Rodada 27')

In [92]:
# Rodada 28

tabela1['Rodada 28'] = pd.Series(dtype='int')
tabela2['Rodada 28'] = pd.Series(dtype='int')
tabela3['Rodada 28'] = pd.Series(dtype='int')
tabela4['Rodada 28'] = pd.Series(dtype='int')
tabela5['Rodada 28'] = pd.Series(dtype='int')
tabela6['Rodada 28'] = pd.Series(dtype='int')
tabela7['Rodada 28'] = pd.Series(dtype='int')
tabela8['Rodada 28'] = pd.Series(dtype='int')
tabela9['Rodada 28'] = pd.Series(dtype='int')
tabela10['Rodada 28'] = pd.Series(dtype='int')
df['Rodada 28'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Athletico Paranaense', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Goias', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Fluminense', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Gremio', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Cruzeiro', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Vasco da Gama', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Palmeiras', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Fortaleza', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Santos', 'Rodada 28')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'América Mineiro', 'Rodada 28')

In [93]:
# Rodada 29

tabela1['Rodada 29'] = pd.Series(dtype='int')
tabela2['Rodada 29'] = pd.Series(dtype='int')
tabela3['Rodada 29'] = pd.Series(dtype='int')
tabela4['Rodada 29'] = pd.Series(dtype='int')
tabela5['Rodada 29'] = pd.Series(dtype='int')
tabela6['Rodada 29'] = pd.Series(dtype='int')
tabela7['Rodada 29'] = pd.Series(dtype='int')
tabela8['Rodada 29'] = pd.Series(dtype='int')
tabela9['Rodada 29'] = pd.Series(dtype='int')
tabela10['Rodada 29'] = pd.Series(dtype='int')
df['Rodada 29'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Bahia', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Atlético Mineiro', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'América Mineiro', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Internacional', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Botafogo', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Flamengo', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Corinthians', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Goias', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'São Paulo', 'Rodada 29')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Coritiba', 'Rodada 29')

In [94]:
# Rodada 30

tabela1['Rodada 30'] = pd.Series(dtype='int')
tabela2['Rodada 30'] = pd.Series(dtype='int')
tabela3['Rodada 30'] = pd.Series(dtype='int')
tabela4['Rodada 30'] = pd.Series(dtype='int')
tabela5['Rodada 30'] = pd.Series(dtype='int')
tabela6['Rodada 30'] = pd.Series(dtype='int')
tabela7['Rodada 30'] = pd.Series(dtype='int')
tabela8['Rodada 30'] = pd.Series(dtype='int')
tabela9['Rodada 30'] = pd.Series(dtype='int')
tabela10['Rodada 30'] = pd.Series(dtype='int')
df['Rodada 30'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'São Paulo', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Bragantino', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Cruzeiro', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Coritiba', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Cuiabá', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Santos', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Bahia', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Gremio', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Fluminense', 'Rodada 30')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Vasco da Gama', 'Rodada 30')

In [95]:
# Rodada 31

tabela1['Rodada 31'] = pd.Series(dtype='int')
tabela2['Rodada 31'] = pd.Series(dtype='int')
tabela3['Rodada 31'] = pd.Series(dtype='int')
tabela4['Rodada 31'] = pd.Series(dtype='int')
tabela5['Rodada 31'] = pd.Series(dtype='int')
tabela6['Rodada 31'] = pd.Series(dtype='int')
tabela7['Rodada 31'] = pd.Series(dtype='int')
tabela8['Rodada 31'] = pd.Series(dtype='int')
tabela9['Rodada 31'] = pd.Series(dtype='int')
tabela10['Rodada 31'] = pd.Series(dtype='int')
df['Rodada 31'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'América Mineiro', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Bragantino', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Athletico Paranaense', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Fortaleza', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Santos', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Gremio', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Fluminense', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Vasco da Gama', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Cruzeiro', 'Rodada 31')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Palmeiras', 'Rodada 31')

In [96]:
# Rodada 32

tabela1['Rodada 32'] = pd.Series(dtype='int')
tabela2['Rodada 32'] = pd.Series(dtype='int')
tabela3['Rodada 32'] = pd.Series(dtype='int')
tabela4['Rodada 32'] = pd.Series(dtype='int')
tabela5['Rodada 32'] = pd.Series(dtype='int')
tabela6['Rodada 32'] = pd.Series(dtype='int')
tabela7['Rodada 32'] = pd.Series(dtype='int')
tabela8['Rodada 32'] = pd.Series(dtype='int')
tabela9['Rodada 32'] = pd.Series(dtype='int')
tabela10['Rodada 32'] = pd.Series(dtype='int')
df['Rodada 32'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Bahia', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Atlético Mineiro', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Cuiabá', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Internacional', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'São Paulo', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Corinthians', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Flamengo', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Botafogo', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Goias', 'Rodada 32')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Athletico Paranaense', 'Rodada 32')

In [97]:
# Rodada 33

tabela1['Rodada 33'] = pd.Series(dtype='int')
tabela2['Rodada 33'] = pd.Series(dtype='int')
tabela3['Rodada 33'] = pd.Series(dtype='int')
tabela4['Rodada 33'] = pd.Series(dtype='int')
tabela5['Rodada 33'] = pd.Series(dtype='int')
tabela6['Rodada 33'] = pd.Series(dtype='int')
tabela7['Rodada 33'] = pd.Series(dtype='int')
tabela8['Rodada 33'] = pd.Series(dtype='int')
tabela9['Rodada 33'] = pd.Series(dtype='int')
tabela10['Rodada 33'] = pd.Series(dtype='int')
df['Rodada 33'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Cuiabá', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Palmeiras', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Vasco da Gama', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Fluminense', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Coritiba', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Santos', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Fortaleza', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Atlético Mineiro', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Gremio', 'Rodada 33')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Bragantino', 'Rodada 33')

In [98]:
# Rodada 34

tabela1['Rodada 34'] = pd.Series(dtype='int')
tabela2['Rodada 34'] = pd.Series(dtype='int')
tabela3['Rodada 34'] = pd.Series(dtype='int')
tabela4['Rodada 34'] = pd.Series(dtype='int')
tabela5['Rodada 34'] = pd.Series(dtype='int')
tabela6['Rodada 34'] = pd.Series(dtype='int')
tabela7['Rodada 34'] = pd.Series(dtype='int')
tabela8['Rodada 34'] = pd.Series(dtype='int')
tabela9['Rodada 34'] = pd.Series(dtype='int')
tabela10['Rodada 34'] = pd.Series(dtype='int')
df['Rodada 34'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'São Paulo', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'América Mineiro', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Fluminense', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Cruzeiro', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Goias', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Botafogo', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Internacional', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Corinthians', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Athletico Paranaense', 'Rodada 34')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Fortaleza', 'Rodada 34')

In [99]:
# Rodada 35

tabela1['Rodada 35'] = pd.Series(dtype='int')
tabela2['Rodada 35'] = pd.Series(dtype='int')
tabela3['Rodada 35'] = pd.Series(dtype='int')
tabela4['Rodada 35'] = pd.Series(dtype='int')
tabela5['Rodada 35'] = pd.Series(dtype='int')
tabela6['Rodada 35'] = pd.Series(dtype='int')
tabela7['Rodada 35'] = pd.Series(dtype='int')
tabela8['Rodada 35'] = pd.Series(dtype='int')
tabela9['Rodada 35'] = pd.Series(dtype='int')
tabela10['Rodada 35'] = pd.Series(dtype='int')
df['Rodada 35'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'Cruzeiro', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Flamengo', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Cuiabá', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Vasco da Gama', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Bragantino', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Coritiba', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'Gremio', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Bahia', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Palmeiras', 'Rodada 35')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Santos', 'Rodada 35')

In [100]:
# Rodada 36

tabela1['Rodada 36'] = pd.Series(dtype='int')
tabela2['Rodada 36'] = pd.Series(dtype='int')
tabela3['Rodada 36'] = pd.Series(dtype='int')
tabela4['Rodada 36'] = pd.Series(dtype='int')
tabela5['Rodada 36'] = pd.Series(dtype='int')
tabela6['Rodada 36'] = pd.Series(dtype='int')
tabela7['Rodada 36'] = pd.Series(dtype='int')
tabela8['Rodada 36'] = pd.Series(dtype='int')
tabela9['Rodada 36'] = pd.Series(dtype='int')
tabela10['Rodada 36'] = pd.Series(dtype='int')
df['Rodada 36'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Goias', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Atlético Mineiro', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Botafogo', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Athletico Paranaense', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'São Paulo', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Fortaleza', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Fluminense', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Internacional', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Corinthians', 'Rodada 36')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'América Mineiro', 'Rodada 36')

In [101]:
# Rodada 37

tabela1['Rodada 37'] = pd.Series(dtype='int')
tabela2['Rodada 37'] = pd.Series(dtype='int')
tabela3['Rodada 37'] = pd.Series(dtype='int')
tabela4['Rodada 37'] = pd.Series(dtype='int')
tabela5['Rodada 37'] = pd.Series(dtype='int')
tabela6['Rodada 37'] = pd.Series(dtype='int')
tabela7['Rodada 37'] = pd.Series(dtype='int')
tabela8['Rodada 37'] = pd.Series(dtype='int')
tabela9['Rodada 37'] = pd.Series(dtype='int')
tabela10['Rodada 37'] = pd.Series(dtype='int')
df['Rodada 37'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Atlético Mineiro', 'São Paulo', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Botafogo', 'Cruzeiro', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Corinthians', 'Internacional', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'América Mineiro', 'Bahia', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fortaleza', 'Goias', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bragantino', 'Coritiba', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Gremio', 'Vasco da Gama', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Flamengo', 'Cuiabá', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Athletico Paranaense', 'Santos', 'Rodada 37')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Palmeiras', 'Fluminense', 'Rodada 37')

In [102]:
# Rodada 38

tabela1['Rodada 38'] = pd.Series(dtype='int')
tabela2['Rodada 38'] = pd.Series(dtype='int')
tabela3['Rodada 38'] = pd.Series(dtype='int')
tabela4['Rodada 38'] = pd.Series(dtype='int')
tabela5['Rodada 38'] = pd.Series(dtype='int')
tabela6['Rodada 38'] = pd.Series(dtype='int')
tabela7['Rodada 38'] = pd.Series(dtype='int')
tabela8['Rodada 38'] = pd.Series(dtype='int')
tabela9['Rodada 38'] = pd.Series(dtype='int')
tabela10['Rodada 38'] = pd.Series(dtype='int')
df['Rodada 38'] = pd.Series(dtype='int')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Internacional', 'Botafogo', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Coritiba', 'Corinthians', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Fluminense', 'Gremio', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cruzeiro', 'Palmeiras', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Bahia', 'Atlético Mineiro', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Cuiabá', 'Athletico Paranaense', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Goias', 'América Mineiro', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Santos', 'Fortaleza', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'São Paulo', 'Flamengo', 'Rodada 38')
times_odds(df, tabela1, tabela2, tabela3, tabela4, tabela5, tabela6, tabela7, tabela8, tabela9, tabela10, 'Vasco da Gama', 'Bragantino', 'Rodada 38')

# Tabela 1 completa

In [115]:
tabela1['Final'] = tabela1.sum(axis=1)
tabela1 = tabela1.sort_values(by='Final', ascending=False)
tabela1.fillna(value = 0, inplace = True)

tabela1.head(20)

<ipython-input-115-76d17b41f036>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela1['Final'] = tabela1.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
0,Palmeiras,1.0,3.0,3.0,0.0,3.0,3.0,1.0,1.0,3.0,...,3.0,3.0,3.0,1.0,1.0,1.0,3.0,3.0,0.0,73.0
18,Gremio,3.0,1.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,...,0.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,73.0
16,Flamengo,3.0,0.0,1.0,0.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,70.0
11,Internacional,0.0,3.0,3.0,3.0,3.0,1.0,0.0,3.0,1.0,...,3.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,61.0
19,Santos,0.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,...,0.0,0.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,60.0
5,São Paulo,3.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0,0.0,...,0.0,3.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,59.0
12,Atlético Mineiro,3.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,1.0,3.0,0.0,0.0,1.0,1.0,59.0
14,Corinthians,1.0,1.0,0.0,3.0,1.0,3.0,0.0,3.0,1.0,...,3.0,1.0,3.0,1.0,0.0,3.0,3.0,1.0,3.0,59.0
15,Cruzeiro,1.0,1.0,3.0,1.0,0.0,3.0,1.0,0.0,0.0,...,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,3.0,55.0
3,Fluminense,1.0,3.0,3.0,0.0,3.0,3.0,1.0,0.0,3.0,...,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,55.0


# Tabela 2 completa

In [117]:
tabela2['Final'] = tabela2.sum(axis=1)
tabela2 = tabela2.sort_values(by='Final', ascending=False)
tabela2.fillna(value = 0, inplace = True)

tabela2.head(20)

<ipython-input-117-9772f7eb538d>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela2['Final'] = tabela2.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
0,Palmeiras,1.0,3.0,3.0,0.0,3.0,3.0,1.0,1.0,3.0,...,3.0,3.0,3.0,1.0,1.0,1.0,3.0,3.0,0.0,146.0
18,Gremio,3.0,1.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,...,0.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,146.0
16,Flamengo,3.0,0.0,1.0,0.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,140.0
11,Internacional,0.0,3.0,3.0,3.0,3.0,1.0,0.0,3.0,1.0,...,3.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,122.0
19,Santos,0.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,...,0.0,0.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,120.0
5,São Paulo,3.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0,0.0,...,0.0,3.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,118.0
12,Atlético Mineiro,3.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,1.0,3.0,0.0,0.0,1.0,1.0,118.0
14,Corinthians,1.0,1.0,0.0,3.0,1.0,3.0,0.0,3.0,1.0,...,3.0,1.0,3.0,1.0,0.0,3.0,3.0,1.0,3.0,118.0
15,Cruzeiro,1.0,1.0,3.0,1.0,0.0,3.0,1.0,0.0,0.0,...,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,3.0,110.0
3,Fluminense,1.0,3.0,3.0,0.0,3.0,3.0,1.0,0.0,3.0,...,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,110.0


# Tabela 3 completa

In [118]:
tabela3['Final'] = tabela3.sum(axis=1)
tabela3 = tabela3.sort_values(by='Final', ascending=False)
tabela3.fillna(value = 0, inplace = True)

tabela3.head(20)

<ipython-input-118-d76cc1cc539b>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela3['Final'] = tabela3.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
15,Cruzeiro,3.0,3.0,3.0,3.0,1.0,3.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,1.0,78.0
0,Palmeiras,3.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,3.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,1.0,71.0
14,Corinthians,0.0,0.0,1.0,3.0,3.0,3.0,1.0,1.0,3.0,...,3.0,0.0,3.0,3.0,0.0,3.0,3.0,0.0,3.0,69.0
19,Santos,1.0,3.0,1.0,0.0,3.0,3.0,3.0,3.0,3.0,...,0.0,1.0,3.0,3.0,1.0,3.0,1.0,3.0,1.0,69.0
16,Flamengo,3.0,1.0,0.0,1.0,3.0,3.0,1.0,3.0,3.0,...,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,65.0
18,Gremio,1.0,0.0,3.0,3.0,1.0,3.0,3.0,1.0,1.0,...,0.0,0.0,3.0,1.0,3.0,3.0,3.0,3.0,1.0,64.0
5,São Paulo,1.0,3.0,1.0,1.0,3.0,0.0,3.0,3.0,1.0,...,1.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,3.0,63.0
12,Atlético Mineiro,1.0,0.0,3.0,1.0,3.0,0.0,0.0,3.0,1.0,...,3.0,3.0,3.0,0.0,3.0,0.0,0.0,3.0,0.0,61.0
11,Internacional,0.0,1.0,3.0,1.0,3.0,3.0,0.0,3.0,0.0,...,3.0,3.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,58.0
3,Fluminense,0.0,3.0,3.0,1.0,1.0,3.0,0.0,1.0,1.0,...,0.0,1.0,1.0,3.0,0.0,3.0,1.0,0.0,1.0,57.0


# Tabela 4 completa

In [119]:
tabela4['Final'] = tabela4.sum(axis=1)
tabela4 = tabela4.sort_values(by='Final', ascending=False)
tabela4.fillna(value = 0, inplace = True)

tabela4.head(20)

<ipython-input-119-c23d3132f701>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela4['Final'] = tabela4.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
5,São Paulo,3.0,1.0,1.0,0.0,3.0,1.0,3.0,3.0,3.0,...,0.0,1.0,3.0,3.0,0.0,3.0,3.0,1.0,3.0,70.0
11,Internacional,0.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,...,3.0,3.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,66.0
19,Santos,0.0,3.0,0.0,0.0,3.0,1.0,1.0,3.0,1.0,...,1.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,65.0
14,Corinthians,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,62.0
18,Gremio,3.0,0.0,3.0,3.0,0.0,3.0,1.0,0.0,0.0,...,0.0,0.0,1.0,3.0,1.0,3.0,1.0,3.0,0.0,60.0
0,Palmeiras,1.0,1.0,0.0,3.0,3.0,0.0,1.0,1.0,0.0,...,3.0,3.0,1.0,3.0,1.0,0.0,3.0,3.0,0.0,59.0
4,Botafogo,0.0,0.0,1.0,3.0,1.0,0.0,1.0,3.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,3.0,57.0
15,Cruzeiro,1.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,3.0,...,3.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,3.0,57.0
16,Flamengo,3.0,0.0,1.0,0.0,3.0,3.0,0.0,3.0,3.0,...,1.0,0.0,3.0,0.0,1.0,3.0,1.0,3.0,0.0,55.0
3,Fluminense,1.0,3.0,1.0,3.0,3.0,3.0,1.0,0.0,3.0,...,1.0,3.0,0.0,0.0,1.0,3.0,0.0,0.0,3.0,53.0


# Tabela 5 completa

In [121]:
tabela5['Final'] = tabela5.sum(axis=1)
tabela5 = tabela5.sort_values(by='Final', ascending=False)
tabela5.fillna(value = 0, inplace = True)
tabela5.head(20)

<ipython-input-121-339f66c3317c>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela5['Final'] = tabela5.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
11,Internacional,0.0,3.0,1.0,0.0,1.0,0.0,3.0,1.0,3.0,...,3.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,3.0,71.0
0,Palmeiras,0.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,...,3.0,1.0,0.0,1.0,0.0,1.0,3.0,1.0,0.0,69.0
16,Flamengo,3.0,0.0,3.0,1.0,3.0,3.0,3.0,1.0,1.0,...,3.0,3.0,3.0,1.0,0.0,3.0,0.0,3.0,3.0,69.0
18,Gremio,1.0,3.0,3.0,3.0,0.0,1.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,3.0,61.0
3,Fluminense,3.0,1.0,3.0,0.0,0.0,3.0,3.0,1.0,1.0,...,0.0,0.0,3.0,3.0,3.0,0.0,3.0,1.0,0.0,59.0
14,Corinthians,3.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,3.0,0.0,0.0,3.0,0.0,3.0,3.0,1.0,0.0,59.0
7,Bahia,0.0,3.0,3.0,3.0,1.0,0.0,3.0,1.0,0.0,...,0.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,0.0,58.0
8,Athletico Paranaense,3.0,1.0,3.0,1.0,1.0,3.0,1.0,0.0,3.0,...,0.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,0.0,56.0
5,São Paulo,0.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,...,3.0,3.0,0.0,3.0,3.0,0.0,0.0,3.0,0.0,55.0
12,Atlético Mineiro,3.0,0.0,0.0,0.0,3.0,3.0,3.0,0.0,3.0,...,3.0,3.0,1.0,0.0,1.0,1.0,3.0,0.0,3.0,54.0


# Tabela 6 completa

In [122]:
tabela6['Final'] = tabela6.sum(axis=1)
tabela6 = tabela6.sort_values(by='Final', ascending=False)
tabela6.fillna(value = 0, inplace = True)

tabela6.head(20)

<ipython-input-122-afab645fbf43>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela6['Final'] = tabela6.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
14,Corinthians,3.0,3.0,0.0,1.0,0.0,3.0,0.0,3.0,3.0,...,3.0,1.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,76.0
16,Flamengo,3.0,1.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,...,0.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,69.0
18,Gremio,0.0,3.0,3.0,3.0,1.0,0.0,3.0,3.0,3.0,...,1.0,1.0,3.0,3.0,0.0,3.0,0.0,1.0,3.0,68.0
19,Santos,3.0,1.0,3.0,3.0,3.0,0.0,3.0,1.0,0.0,...,0.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,3.0,64.0
0,Palmeiras,0.0,3.0,3.0,0.0,1.0,3.0,0.0,3.0,3.0,...,1.0,0.0,3.0,3.0,0.0,0.0,3.0,3.0,1.0,61.0
3,Fluminense,0.0,3.0,3.0,0.0,3.0,3.0,3.0,0.0,3.0,...,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,60.0
11,Internacional,1.0,1.0,0.0,3.0,3.0,1.0,0.0,3.0,3.0,...,3.0,3.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0,58.0
12,Atlético Mineiro,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,...,3.0,3.0,0.0,3.0,0.0,0.0,0.0,3.0,3.0,57.0
5,São Paulo,3.0,3.0,3.0,0.0,0.0,0.0,3.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,56.0
15,Cruzeiro,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,3.0,3.0,3.0,3.0,3.0,0.0,1.0,1.0,55.0


# Tabela 7 completa

In [123]:
tabela7['Final'] = tabela7.sum(axis=1)
tabela7 = tabela7.sort_values(by='Final', ascending=False)
tabela7.fillna(value = 0, inplace = True)

tabela7.head(20)

<ipython-input-123-8bd0a5af3cec>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela7['Final'] = tabela7.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
5,São Paulo,3.0,1.0,3.0,3.0,3.0,0.0,3.0,0.0,3.0,...,0.0,3.0,0.0,3.0,3.0,1.0,3.0,3.0,1.0,71.0
16,Flamengo,3.0,0.0,0.0,3.0,1.0,3.0,0.0,1.0,1.0,...,3.0,0.0,3.0,3.0,0.0,3.0,3.0,0.0,1.0,65.0
0,Palmeiras,3.0,1.0,0.0,3.0,3.0,0.0,3.0,0.0,0.0,...,0.0,3.0,3.0,0.0,3.0,0.0,1.0,1.0,0.0,64.0
14,Corinthians,3.0,0.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,...,1.0,3.0,3.0,3.0,0.0,1.0,3.0,3.0,3.0,63.0
12,Atlético Mineiro,3.0,1.0,0.0,1.0,3.0,3.0,3.0,3.0,3.0,...,3.0,1.0,3.0,0.0,3.0,0.0,0.0,0.0,3.0,62.0
18,Gremio,3.0,1.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,...,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,62.0
19,Santos,0.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,3.0,...,1.0,3.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,62.0
4,Botafogo,0.0,0.0,3.0,1.0,0.0,1.0,3.0,3.0,0.0,...,3.0,0.0,3.0,0.0,3.0,0.0,3.0,3.0,1.0,60.0
8,Athletico Paranaense,3.0,0.0,3.0,0.0,0.0,3.0,3.0,3.0,3.0,...,3.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,57.0
1,Cuiabá,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,3.0,...,0.0,0.0,0.0,3.0,3.0,1.0,3.0,3.0,3.0,55.0


# Tabela 8 completa

In [124]:
tabela8['Final'] = tabela8.sum(axis=1)
tabela8 = tabela8.sort_values(by='Final', ascending=False)
tabela8.fillna(value = 0, inplace = True)

tabela8.head(20)

<ipython-input-124-499965922695>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela8['Final'] = tabela8.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
19,Santos,0.0,3.0,3.0,1.0,1.0,3.0,3.0,3.0,3.0,...,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,72.0
0,Palmeiras,3.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,3.0,3.0,3.0,1.0,3.0,3.0,70.0
15,Cruzeiro,3.0,3.0,0.0,1.0,3.0,3.0,3.0,0.0,3.0,...,0.0,0.0,3.0,3.0,0.0,3.0,3.0,3.0,0.0,68.0
8,Athletico Paranaense,3.0,1.0,0.0,3.0,1.0,3.0,0.0,3.0,3.0,...,0.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,3.0,67.0
5,São Paulo,3.0,3.0,3.0,0.0,0.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,65.0
18,Gremio,3.0,0.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,0.0,0.0,3.0,3.0,3.0,1.0,3.0,3.0,0.0,63.0
16,Flamengo,3.0,3.0,0.0,0.0,3.0,1.0,0.0,3.0,3.0,...,3.0,0.0,3.0,0.0,3.0,1.0,3.0,1.0,1.0,62.0
14,Corinthians,0.0,0.0,3.0,3.0,3.0,0.0,3.0,3.0,3.0,...,3.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,3.0,61.0
11,Internacional,0.0,0.0,3.0,3.0,1.0,3.0,3.0,3.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,57.0
12,Atlético Mineiro,0.0,0.0,3.0,0.0,3.0,0.0,0.0,3.0,0.0,...,1.0,0.0,3.0,3.0,3.0,1.0,0.0,3.0,3.0,57.0


# Tabela 9 completa

In [125]:
tabela9['Final'] = tabela9.sum(axis=1)
tabela9 = tabela9.sort_values(by='Final', ascending=False)
tabela9.fillna(value = 0, inplace = True)

tabela9.head(20)

<ipython-input-125-b8b2715bd650>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela9['Final'] = tabela9.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
18,Gremio,0.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,...,3.0,0.0,3.0,3.0,3.0,3.0,3.0,0.0,1.0,72.0
3,Fluminense,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,3.0,...,0.0,0.0,3.0,1.0,3.0,3.0,3.0,3.0,1.0,71.0
16,Flamengo,1.0,0.0,3.0,3.0,3.0,3.0,1.0,0.0,0.0,...,3.0,3.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,64.0
4,Botafogo,0.0,3.0,0.0,3.0,1.0,0.0,0.0,3.0,3.0,...,3.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,62.0
11,Internacional,1.0,3.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,...,3.0,1.0,3.0,1.0,1.0,3.0,0.0,3.0,1.0,62.0
12,Atlético Mineiro,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,0.0,0.0,3.0,0.0,0.0,61.0
15,Cruzeiro,3.0,1.0,0.0,3.0,0.0,3.0,3.0,3.0,0.0,...,3.0,3.0,0.0,3.0,3.0,0.0,1.0,1.0,0.0,57.0
14,Corinthians,0.0,3.0,1.0,0.0,1.0,3.0,1.0,3.0,0.0,...,0.0,1.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,57.0
0,Palmeiras,0.0,1.0,1.0,0.0,0.0,3.0,1.0,1.0,3.0,...,3.0,3.0,3.0,0.0,1.0,3.0,3.0,0.0,3.0,56.0
2,América Mineiro,0.0,0.0,3.0,1.0,3.0,0.0,3.0,0.0,3.0,...,0.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,3.0,55.0


# Tabela 10 completa

In [126]:
tabela10['Final'] = tabela10.sum(axis=1)
tabela10 = tabela10.sort_values(by='Final', ascending=False)
tabela10.fillna(value = 0, inplace = True)

tabela10.head(20)

<ipython-input-126-d988a51dfa12>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tabela10['Final'] = tabela10.sum(axis=1)


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,...,Rodada 30,Rodada 31,Rodada 32,Rodada 33,Rodada 34,Rodada 35,Rodada 36,Rodada 37,Rodada 38,Final
11,Internacional,1.0,0.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,1.0,3.0,3.0,3.0,3.0,0.0,0.0,64.0
16,Flamengo,3.0,3.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,...,3.0,3.0,1.0,0.0,3.0,3.0,0.0,3.0,0.0,62.0
19,Santos,1.0,3.0,3.0,0.0,3.0,0.0,0.0,3.0,0.0,...,1.0,0.0,3.0,3.0,3.0,0.0,3.0,1.0,0.0,61.0
15,Cruzeiro,0.0,3.0,0.0,3.0,0.0,1.0,3.0,0.0,3.0,...,3.0,1.0,1.0,3.0,3.0,1.0,0.0,0.0,3.0,61.0
3,Fluminense,0.0,3.0,3.0,0.0,3.0,3.0,1.0,3.0,3.0,...,3.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,60.0
14,Corinthians,3.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,...,1.0,3.0,0.0,3.0,3.0,1.0,3.0,3.0,0.0,59.0
12,Atlético Mineiro,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,...,0.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0,0.0,59.0
0,Palmeiras,0.0,3.0,3.0,1.0,1.0,1.0,3.0,3.0,1.0,...,0.0,1.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,59.0
5,São Paulo,3.0,0.0,3.0,0.0,1.0,0.0,3.0,1.0,0.0,...,0.0,1.0,3.0,1.0,0.0,3.0,3.0,1.0,3.0,59.0
9,Goias,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,...,0.0,0.0,3.0,0.0,1.0,1.0,3.0,3.0,0.0,55.0
